In [144]:
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [145]:
SPOTIPY_CLIENT_ID = "e1391d7dc34a49879436c47fa08a8474"
SPOTIPY_CLIENT_SECRET = "811c492753ef4ca6840d4de1019c33ae"
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID,
                                                           client_secret=SPOTIPY_CLIENT_SECRET))

In [273]:
spotify_data = pd.read_csv('../data/data.csv')

In [ ]:
import ast
artists_saved = {}
artist_genre = []

In [466]:

def print_info(i, track_id):
    print()
    print('track id', track_id)
    print('item no ' + str(i))
    print('porcentage ' + str(i/174389))

for i in range(len(artist_genre), len(spotify_data)):
    
    track_id = spotify_data.iloc[i].id
    track = sp.track(track_id)
    
    print_info(i, track_id)
    
    if len(track['artists']) == 0:
        # chequeo si la API devuelve un track con artistas, 
        # sigo al proximo elemento si esta vacio
        artist_genre.append([])
        continue
    
    else:   
        
        aux_genres = []
        
        for artist in track['artists']:

            try:
                # chequeo si es un artista ya fetcheado, 
                # asi no hago un llamado a la API devuelta
                
                artist_genre_saved = artists_saved[artist['id']]
                for s in artist_genre_saved:
                    aux_genres.append(s)
                continue
            except:
                None

            art_id = artist['id']
            artist_detail = sp.artist(art_id)

            if artist_detail['genres'] != []:
                #busco artista en API por ID
                
                artists_saved[art_id] = artist_detail['genres']
                
                for g in artist_detail['genres']:
                    aux_genres.append(g)
            
            else:
    
                #busco artista en API por NOMBRE 
        
                artists_name = str(spotify_data.iloc[i].artists)
                artists_name = ast.literal_eval(artists_name)

                for artist_name in artists_name:
                    results = sp.search(q='artist:' + artist_name, type='artist')
                    items = results['artists']['items']
                    try:
                        for g in items[0]['genres']:
                            aux_genres.append(g)
                            
                        artists_saved[art_id] = items[0]['genres']
                    except:
                        None
                
        aux_genres = list(set(aux_genres))
        artist_genre.append(aux_genres)
        print(aux_genres)
            

In [467]:
import pickle

with open("artist_genre_" + str(len(artist_genre)) + ".txt", "wb") as fp:   #Pickling
    pickle.dump(artist_genre, fp)

In [468]:
with open("artist_genre_" + str(len(artist_genre)) + ".txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)

In [478]:
spotify_data['artist_genre'] = artist_genre

174389

In [480]:
spotify_data.to_csv('../data/data_with_genres.csv')

# User-based Recommender System

1. How do you determine which users or items are similar to one another?
2. Given that you know which users are similar, how do you determine the rating that a user would give to an item based on the ratings of similar users?
3. How do you measure the accuracy of the ratings you calculate?

# step 1 - How to Find Similar Users on the Basis of Ratings

In [292]:
import pandas as pd
import numpy as np
from scipy import spatial


In [293]:
users = []
for i in range(0, 20):
    user = pd.read_csv('../data/users/user_' + str(i) + '.csv')
    users.append(user)


In [304]:
all_songs = []
for user in users:
    user_songs = user.id.tolist()
    all_songs.extend(user_songs)

print(len(all_songs))
all_songs = list(set(all_songs))
print(len(all_songs))


880
859


sucede que los usuarios generados casi que escucharon todas canciones distintas, entonces no pareciera haber usuario parecidos.

In [295]:
artists = []
for u in users:
    user_artists = u.artists.tolist()
    artists.extend(user_artists)

print(len(artists))
print(len(list(set(artists))))

880
423


parece que a pesar de escuchar canciones distintas, si hay mas usuarios que escuchan al mismo artista(aunque canciones distintas)

se podria hacer un calculo de que tan popular es un artisa para un usuario, basado en un promedio de la popularidad de cancoines que escucho de ese artista. Entonces, la similitud de usuarios se calcularia en base a que tan popular consideran artistas y no canciones.

In [296]:

users_songs_matrix = []
for u in users:
    
    user_songs = []
    
    for song_id in all_songs:
        user_song = u[u.id == song_id]
        user_song = user_song.popularity_per_user.tolist()
        
        if user_song == []:
            user_songs.append(None)
        else:
            user_songs.append(user_song[0])
    
    users_songs_matrix.append(user_songs)

#creating the users_songs_matrix, but its a matrix which if the user doesnt listen to a song,
#then this position will have a None value

In [297]:
#replacing the users_songs_matrix None values with each "user popularity mean" value.
#Each song will be more or less popular for each user. So we calculate the mean of popularity of each user.
#Each user will have a different popularity mean value.

mean_vector = []
for i, all_songs in enumerate(users_songs_matrix):
    u = users[i]
    mean_vector.append(np.mean(u.popularity_per_user.tolist()))

#Above we calculated each "user popularity mean"

aux = []
for i,mean_value in enumerate(mean_vector):
    user_songs = users_songs_matrix[i]
    aux.append([mean_value if v is None else v for v in user_songs])

#Above we are replacing None values according to each "user popularity mean"

users_songs_matrix = aux

aux = []
for user_songs in users_songs_matrix:
    user_songs = user_songs - np.mean(user_songs)
    aux.append(user_songs)

users_songs_matrix = aux

#Above we did kind of the same. We have users_songs_matrix with None values replaced by mean vector.
#But we did one more step. This is centering values, so we get the user mean value again and we substracted it 
#to the song user popularity value.

#
#
# - We already have our users_songs_matrix ready !!!
#
#

#### Why we are doing this? Since we'll use cosine distance. There is something that we should handle that is "tough raters".

Notice that users A and B are considered absolutely similar in the cosine similarity metric despite having different ratings. This is actually a common occurrence in the real world, and the users like the user A are what you can call tough raters. An example would be a movie critic who always gives out ratings lower than the average, but the rankings of the items in their list would be similar to the Average raters like B.

[...]

Continue reading on https://realpython.com/build-recommendation-engine-collaborative-filtering/#memory-based

In [327]:

user_to_make_recommendation_index = 5
users.insert(0, users.pop(user_to_make_recommendation_index))
users_songs_matrix.insert(0, users_songs_matrix.pop(user_to_make_recommendation_index))


In [328]:
# pivot_user is the user for which we'll look for songs to recommned
# similar_users are everyone else.
pivot_user_df = users[0]
pivot_user = users_songs_matrix[0]

similar_users = users_songs_matrix[1:]


In [329]:
df=pd.DataFrame(users_songs_matrix, columns=all_songs)
print(df.shape)
df.head()

(20, 859)


,5k64eI1sCsJiFjkM2HTFi3,161R9ZniP8Pa90s1wAkaJx,2H5I2jH11KCl7mT1DiNpIO,73OX8GdpOeGzKC6OvGSbsv,3veSDvjSgV1Ive1MxGD5AY,61EpvDHnicRLbkzmgjyFfK,1EfdsCcge41LcPtSpkZJ50,4kRMsLX7bJqjIfK44qJ9h6,2UnFiaDT4K42B75CAmYnzY,4sRwDJ4BQqoff1c8z01Paw,...,61kS94p8ZUGsXWpUtIevLl,1AHf5FSofKcUw8tyKkccKF,0cC2WGbgZvIxhVCIrxbfO0,4uLtvmkHB77wl5jjhrN6g9,4XWItTtGvpXeCFBduYg0F6,26A8IIHFL1Y9GufqR3I65T,1WIKP9uknZ9aIczo7b7FpL,1V2AVM79zzwU01jNFahRwP,62UWa981fvj8tcgv92J8Bm,2mj9pxHhXaBA9T57kufWPZ
0,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,...,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03,5.093132e-03
1,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-3.950000e+00,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,...,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16,-8.881784e-16
2,3.526630e-01,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,...,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03,-1.503686e-03
3,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,...,8.881784e-16,8.881784e-16,8.881784e-16,8.881784e-16,4.068182e+00,8.881784e-16,8.881784e-16,1.068182e+00,8.881784e-16,8.881784e-16
4,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,...,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03,-1.843052e+00,-2.143084e-03,-2.143084e-03,-2.143084e-03,-2.143084e-03


In [330]:
users_similarities = []
for user_index, s_u in enumerate(similar_users):
    user_similarity = spatial.distance.cosine(pivot_user, s_u)
    users_similarities.append((user_index, user_similarity))

print(users_similarities)

[(0, 1.0), (1, 0.9999814077485696), (2, 1.0), (3, 0.9999749669583909), (4, 0.9999756005005788), (5, 1.0), (6, 1.0), (7, 1.0000042063060248), (8, 1.0000160235335387), (9, 1.0), (10, 0.9999774416783246), (11, 1.0), (12, 1.000016822051019), (13, 0.999966654944389), (14, 0.999979192452423), (15, 1.008484322535909), (16, 1.0), (17, 1.0), (18, 1.0)]


In [331]:
# we should choose the TOP N most similar users (our similarity list) and continue.

n = 5
users_similarities = sorted(similarities, key=lambda tup: tup[1])
users_similarities = users_similarities[:n]
users_similarities

[(1, 0.9999749669583909),
 (12, 0.9999935728303224),
 (8, 0.9999938779184078),
 (7, 0.9999983929044974),
 (2, 1.0)]

# step 2 - How to Calculate the Ratings

In [332]:
# We now have our N most similar users. We just need to take the songs that the "pivot user" didnt listen 
# and based on the N most similar users, recommend songs to the pivot user. 

In [333]:
users_songs_matrix = users_songs_matrix[1:] #removing pivot user

In [334]:
pivot_user_songs = pivot_user_df.id.tolist()
songs_to_recommend = []

for s in all_songs:
    if s not in pivot_user_songs:
        songs_to_recommend.append(s)
        
# We're gettting all the songs that the pivot user didnt listen to.
# The idea is understand which song, the pivot user will like more.

In [335]:
songs_like_prob = []

for song_id in songs_to_recommend:
    song_like_popularity = df[1:][song_id].tolist()#skipping pos 0 because is pivot user
    
    aux = []
    for i,_ in users_similarities:
        aux.append(song_like_popularity[i])

    song_like_popularity = aux
    
    #Above we're getting the song popularities of the top N most similar users.
    
    popularity_users_factor = []
    for index, user_song_like_popularity in enumerate(song_like_popularity):
        factor = 1/(users_similarities[index][1])#could be 0??
        popularity_users_factor.append(factor)
    
    # Above we're calculating a popularity factor, because we want to pay more attention to the
    # users that are more similar to the pivot user.
    
    aux = []
    for i1,i2 in zip(popularity_users_factor, song_like_popularity):
        aux.append(i1*i2)
    
    song_like_popularity = aux
    
    #Above we're applying this popular factor to each similar user liked song.
    
    song_like_popularity = np.mean(song_like_popularity) 
    #
    #
    # Since we are calculating the centered cosine. We've subtracted the averange, 
    # so we have negative popularity values... should I need to apply abs() ????
    #
    #
    
    songs_like_prob.append((song_id, song_like_popularity))
    
songs_like_prob = sorted(songs_like_prob, key=lambda tup: tup[1])

https://realpython.com/build-recommendation-engine-collaborative-filtering/#how-to-calculate-the-ratings

In [336]:
songs_like_prob = sorted(songs_like_prob, key=lambda tup: tup[1])
songs_like_prob

[('0ThY4m18BDygljq0qq0phK', -0.9289324085758324),
 ('5ozjHBJ8Y15zggY8uKYZAX', -0.9289324085758324),
 ('2GKZclxupAkK8It2BQ7qfg', -0.9289324085758324),
 ('4uUKgJS8XN3QSFeu9VUda4', -0.9289324085758324),
 ('6FzxGdypdYZJO1pjDHlqML', -0.9289324085758324),
 ('1AGglZcECKcyPu6pJWeez0', -0.9289324085758324),
 ('6MWoRt97mnSTXZhu3ggi9C', -0.7952021394677967),
 ('6wfugRLamFsTRbPcCpNnP7', -0.7952021394677967),
 ('4R2kfaDFhslZEMJqAFNpdd', -0.7952021394677967),
 ('0uHzHWC18T9aQZ3F7q8XR7', -0.7861061178226939),
 ('3TdqevEejQsHSHjjePOhDJ', -0.7861061178226939),
 ('046iRI9dHeqfFTexrRqnQS', -0.7861061178226939),
 ('1xqgz0Hp1DK2nEPujBay1F', -0.7861061178226939),
 ('0WYfplATC0zj9ucvRgR0HR', -0.7861061178226939),
 ('3SrWfIP72ecixAyRlpOZ0C', -0.7747472261013386),
 ('2Sdf841jsePwm0JvQN5Lbx', -0.7747472261013386),
 ('2PD9RuhXRUH9vsizEVhJWH', -0.7747472261013386),
 ('0yndiszlOMgxOu3y2vKZvB', -0.7747472261013386),
 ('45kff9ZQCPauKNSV5uKHu0', -0.7747472261013386),
 ('5MyjPPvGfJLuEVsy8vz98H', -0.7497436867826215),


In [337]:
n_songs_to_recommend = 5
songs_to_recommend_to_user = songs_like_prob[-n_songs_to_recommend:]

In [338]:
aux = []
for song_id, _ in songs_to_recommend_to_user:
    aux.append(song_id)

songs_to_recommend_to_user = aux
songs_to_recommend_to_user

['1Ql45vwLjZCRBkljInQqOT',
 '1ZgYSewKbuI3rNxnI3OdIS',
 '4uznNt16UKupeCqv8t3JJG',
 '62gGK7c1I4oLRzZB7mPZLp',
 '6Th6ND8RQPywiyelecP5Lf']

In [339]:
songs_to_recommend_to_user = spotify_data[spotify_data['id'].isin(songs_to_recommend_to_user)]
songs_to_recommend_to_user

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
11949,0.00533,['AC/DC'],0.532,215533,0.8580,0,6Th6ND8RQPywiyelecP5Lf,0.000002,9,0.7510,-5.023,1,What Do You Do for Money Honey,61,1980-07-25,0.0599,125.698,0.4950,1980
139216,0.96100,['Hope51'],0.345,242471,0.0731,0,1Ql45vwLjZCRBkljInQqOT,0.947000,9,0.1110,-21.429,0,Dreams and future,0,2021-01-19,0.0348,102.560,0.0785,2021
143116,0.96100,['Hope51'],0.345,242471,0.0731,0,1Ql45vwLjZCRBkljInQqOT,0.947000,9,0.1110,-21.429,0,Dreams and future,0,2021-01-19,0.0348,102.560,0.0785,2021
146863,0.92500,['Oscar Peterson'],0.636,177760,0.1240,0,1ZgYSewKbuI3rNxnI3OdIS,0.663000,5,0.1520,-20.627,0,Just A-Sittin' And A-Rockin',12,1960-02-15,0.0421,93.881,0.2370,1960
157988,0.87900,['Night Club Jazz'],0.709,131580,0.2180,0,62gGK7c1I4oLRzZB7mPZLp,0.451000,0,0.0933,-15.266,1,Striking The Year,5,2017-01-23,0.1520,66.341,0.8110,2017
159317,0.27100,['Эрнест Хемингуэй'],0.712,98700,0.1320,1,4uznNt16UKupeCqv8t3JJG,0.000000,11,0.2360,-16.537,0,Часть 63.3 - По ком звонит колокол,0,1940,0.9520,92.767,0.5600,1940


In [ ]:
#TODO: 

#Estoy agarrando el primer usuario del listado y en base a ese usuario, hago todos los calculos.
#No es muy dinamico, como para que se hagan los calculos en base al usuario en la posicion 6 por ejemplo

#Lo que puedo hacer es agarrar el array de "users" (que tiene el df de cada user), poner 1ero al usuario 
#que le quiera hacer las recomendaciones (es decir mover al de la posicion 6 a la posicion 0) y listo,
#sigue todo mi codigo igual

In [ ]:
#TODO:

#En un mapa con reduccion de dimensiones (SVD o TSNE) validar si los 5 temas que estoy recomendando
#estan cerca de los temas que le gustan al usuario. (Esto va a depender si hay usuario que son parecidoas a el)
